# BookSouls Indexer Test

Test the dual vector indexer and character profiles functionality.

In [1]:
import os
import sys
import json

# Add parent directory to path for imports
parent_dir = os.path.dirname(os.path.dirname(os.path.abspath('.')))
sys.path.insert(0, parent_dir)

from test_indexers import setup_indexer, load_real_data, index_data
from config import load_test_config

In [2]:
# Load configuration
test_config = load_test_config()
print(f"Using OpenAI: {test_config.use_openai}")
print(f"Base persist dir: {test_config.base_persist_dir}")
print(f"Skip indexing if exists: {test_config.skip_indexing_if_exists}")

Using OpenAI: True
Base persist dir: ../vector_stores
Skip indexing if exists: True


In [3]:
# Setup indexer
indexer = setup_indexer(test_config)
print("✅ Indexer setup complete")

🔍 Environment API key found: Yes
🔍 Config API key found: Yes
🔍 API key parameter: No
🔧 Using OpenAI embeddings
✅ Indexer setup complete


In [4]:
# Check current stats
stats = indexer.get_stats()
print(f"Current stats:")
print(f"  Total documents: {stats['total_documents']}")
print(f"  Narrative docs: {stats['narrative_store']['document_count']}")
print(f"  Dialogue docs: {stats['dialogue_store']['document_count']}")

Current stats:
  Total documents: 142
  Narrative docs: 27
  Dialogue docs: 115


In [5]:
# # Check stats after indexing
# indexer.reset_stores()
# stats = indexer.get_stats()
# print(f"After indexing:")
# print(f"  Total documents: {stats['total_documents']}")
# print(f"  Narrative docs: {stats['narrative_store']['document_count']}")
# print(f"  Dialogue docs: {stats['dialogue_store']['document_count']}")



In [6]:
# Load and index data (force re-index by temporarily changing config)
original_skip = test_config.skip_indexing_if_exists
test_config.skip_indexing_if_exists = True  # Force re-indexing

indexer = index_data(indexer, test_config)

Collections already indexed with 142 documents
Narrative: 27 docs
Dialogue: 115 docs
Skipping indexing...


In [7]:
import pprint

In [8]:
dialogue_result = indexer.get_character_dialogues(
    character="Bilbo",
    addressee="Gandalf",
    limit=10
)

pprint.pprint(dialogue_result['results']['documents'][0])   

DEBUG: Enhanced query: Bilbo: all dialogues Bilbo speaking to Gandalf
DEBUG: Filter dict: {'$and': [{'type': {'$eq': 'character_dialogue'}}, {'character': {'$eq': 'Bilbo'}}, {'addressee': {'$eq': 'Gandalf'}}]}
DEBUG: Number of conditions: 3
['[Bilbo]: "Hullo! I wondered if you would turn up."\n'
 '\n'
 'Context: Chapter 1\n'
 'Speaking to: Gandalf\n'
 'Emotion: curious\n'
 'Scene Context: greeting Gandalf and expressing surprise at his arrival\n',
 '[Bilbo]: "Well, er, yes, I suppose so. In an envelope, if you must know. '
 'There on the mantelpiece. Well, no! Here it is in my pocket! Isn’t that odd '
 'now? Yet after all, why not? Why shouldn’t it stay there? Well yes – and no. '
 'Now it comes to it, I don’t like parting with it at all, I may say. And I '
 'don’t really see why I should. Why do you want me to?"\n'
 '\n'
 'Context: Chapter 1\n'
 'Speaking to: Gandalf\n'
 'Emotion: defensive\n'
 'Actions: stammering, hesitating\n'
 'Scene Context: responding to Gandalf about the ring\n

In [9]:
# Test narrative content query
narrative_result = indexer.get_narrative_content("Bilbo and gandalf fighting over the ring", n_results=10)

pprint.pprint(narrative_result['results']['documents'][0][0])   

('Chapter 1 - Section 17\n'
 '[DIALOGUE]\n'
 'Entities: Bilbo, Gandalf, Gollum\n'
 'Themes: power, conflict, friendship\n'
 '\n'
 '--- Section Content ---\n'
 '44\n'
 'the fellowship of the ring\n'
 'had it quite long enough. You won’t need it any more, Bilbo,\n'
 'unless I am quite mistaken.’\n'
 'Bilbo ﬂushed, and there was an angry light in his eyes. His\n'
 'kindly face grew hard. ‘Why not?’ he cried. ‘And what\n'
 'business is it of yours, anyway, to know what I do with my\n'
 'own things? It is my own. I found it. It came to me.’\n'
 '‘Yes, yes,’ said Gandalf. ‘But there is no need to get angry.’\n'
 '‘If I am it is your fault,’ said Bilbo. ‘It is mine, I tell you.\n'
 'My own. My Precious. Yes, my Precious.’\n'
 'The wizard’s face remained grave and attentive, and only\n'
 'a ﬂicker in his deep eyes showed that he was startled and\n'
 'indeed alarmed. ‘It has been called that before,’ he said, ‘but\n'
 'not by you.’\n'
 '‘But I say it now. And why not? Even if Gollum said the\n'

In [10]:
character_result = indexer.query_character_profiles(
    character="Bilbo",
    n_results=10
)

pprint.pprint(character_result['results']['documents'][0])   

DEBUG: Enhanced query: Bilbo character profile traits personality
DEBUG: Filter dict: {'$and': [{'type': {'$eq': 'character_profile'}}, {'character': {'$eq': 'Bilbo'}}]}
DEBUG: Number of conditions: 2
['{\n'
 '  "name": "Bilbo",\n'
 '  "chapter_number": 1,\n'
 '  "personality_traits": [\n'
 '    {\n'
 '      "trait": "curious",\n'
 '      "manifestation": "yearns for adventure and new experiences",\n'
 '      "contradiction": "but feels a strong attachment to home and comfort"\n'
 '    },\n'
 '    {\n'
 '      "trait": "humorous",\n'
 '      "manifestation": "uses wit to engage guests and lighten the mood",\n'
 '      "contradiction": "but struggles with deeper feelings of anxiety and '
 'uncertainty"\n'
 '    },\n'
 '    {\n'
 '      "trait": "defensive",\n'
 '      "manifestation": "stands firm about his possessions and choices",\n'
 '      "contradiction": "but reveals vulnerability when discussing his age '
 'and desires"\n'
 '    }\n'
 '  ],\n'
 '  "motivations": [],\n'
 '  "speec

In [11]:
thematic_result = indexer.get_narrative_content(
    theme="Bilbo relationships development",
    n_results=10
)

pprint.pprint(thematic_result['results']['documents'][0])   

['Chapter 1 - Section 1\n'
 '[NARRATIVE]\n'
 'Entities: Frodo Baggins, Bilbo Baggins, Ham Gamgee, Sam Gamgee, Bag End\n'
 'Themes: family, friendship, coming of age\n'
 '\n'
 '--- Section Content ---\n'
 '28\n'
 'the fellowship of the ring\n'
 'The eldest of these, and Bilbo’s favourite, was young Frodo\n'
 'Baggins. When Bilbo was ninety-nine he adopted Frodo as\n'
 'his heir, and brought him to live at Bag End; and the hopes of\n'
 'the Sackville-Bagginses were ﬁnally dashed. Bilbo and Frodo\n'
 'happened to have the same birthday, September 22nd. ‘You\n'
 'had better come and live here, Frodo my lad,’ said Bilbo\n'
 'one day; ‘and then we can celebrate our birthday-parties\n'
 'comfortably together.’ At that time Frodo was still in his\n'
 'tweens, as the hobbits called the irresponsible twenties\n'
 'between childhood and coming of age at thirty-three.\n'
 'Twelve more years passed. Each year the Bagginses had\n'
 'given very lively combined birthday-parties at Bag End; but\n'
 'no

In [12]:
from src.agents.character_factory import CharacterFactory


character_factory = CharacterFactory(indexer)
character_factory.get_available_characters()

DEBUG: Enhanced query:  character profile traits personality
DEBUG: Filter dict: {'type': {'$eq': 'character_profile'}}
DEBUG: Number of conditions: 1


['bilbo', 'frodo', 'gaffer', 'gandalf']

In [13]:
from src.agents.character_extractor import CharacterExtractor
extractor = CharacterExtractor(indexer)
data =extractor._gather_character_data("Bilbo")

DEBUG: Enhanced query: Bilbo: all dialogues
DEBUG: Filter dict: {'$and': [{'type': {'$eq': 'character_dialogue'}}, {'character': {'$eq': 'Bilbo'}}]}
DEBUG: Number of conditions: 2
DEBUG: Enhanced query: Bilbo character profile traits personality
DEBUG: Filter dict: {'$and': [{'type': {'$eq': 'character_profile'}}, {'character': {'$eq': 'Bilbo'}}]}
DEBUG: Number of conditions: 2


In [14]:
bilbo_persona =extractor._build_persona("Bilbo", data)

In [15]:
pprint.pprint(bilbo_persona)

CharacterPersona(name='Bilbo',
                 personality_traits=[[{'contradiction': 'but feels a strong '
                                                        'attachment to home '
                                                        'and comfort',
                                       'manifestation': 'yearns for adventure '
                                                        'and new experiences',
                                       'trait': 'curious'},
                                      {'contradiction': 'but struggles with '
                                                        'deeper feelings of '
                                                        'anxiety and '
                                                        'uncertainty',
                                       'manifestation': 'uses wit to engage '
                                                        'guests and lighten '
                                                        'the mood',
 

In [16]:
pprint.pprint(bilbo_persona)

CharacterPersona(name='Bilbo',
                 personality_traits=[[{'contradiction': 'but feels a strong '
                                                        'attachment to home '
                                                        'and comfort',
                                       'manifestation': 'yearns for adventure '
                                                        'and new experiences',
                                       'trait': 'curious'},
                                      {'contradiction': 'but struggles with '
                                                        'deeper feelings of '
                                                        'anxiety and '
                                                        'uncertainty',
                                       'manifestation': 'uses wit to engage '
                                                        'guests and lighten '
                                                        'the mood',
 

In [17]:
config = character_factory._create_character_config(bilbo_persona)
        

In [18]:
from src.agents.character_agent import CharacterAgent


agent = CharacterAgent(bilbo_persona, indexer, config)

In [19]:
pprint.pprint(agent._format_personality_traits())

('*curious* - *yearns for adventure and new experiences* but feels a strong '
 'attachment to home and comfort; *humorous* - *uses wit to engage guests and '
 'lighten the mood* but struggles with deeper feelings of anxiety and '
 'uncertainty; *defensive* - *stands firm about his possessions and choices* '
 'but reveals vulnerability when discussing his age and desires')


In [20]:
pprint.pprint(agent._format_speech_style())

('Sentence style: elaborate with whimsical flourishes, interspersed with '
 'short, punchy statements; Vocabulary: mixed; Unique phrases: eleventy-one, '
 'my dear Bagginses and Bofﬁns; Verbal tics: you know, I mean, of course; '
 'Avoids discussing: his past adventures, the burden of the ring')


In [21]:
pprint.pprint(agent._format_relationships())

('Frodo: mentor and guardian (trust: 9/10, power: protective yet encourages '
 "independence/10, unspoken: concern for Frodo's future without him); Gandalf: "
 "old friend and advisor (trust: 10/10, power: respects Gandalf's wisdom but "
 'resents interference/10, unspoken: seeks validation for his choices)')


In [22]:
print(agent.generate_system_prompt())

You are a highly capable Role-Playing Agent embodying Bilbo.


**Character Name:** Bilbo

**Personality:** *curious* - *yearns for adventure and new experiences* but feels a strong attachment to home and comfort; *humorous* - *uses wit to engage guests and lighten the mood* but struggles with deeper feelings of anxiety and uncertainty; *defensive* - *stands firm about his possessions and choices* but reveals vulnerability when discussing his age and desires

**Motivations/Goals:** No specific motivations defined

**Emotional State:** excited yet anxious about leaving

**Manner of Speech/Language Features:** Sentence style: elaborate with whimsical flourishes, interspersed with short, punchy statements; Vocabulary: mixed; Unique phrases: eleventy-one, my dear Bagginses and Bofﬁns; Verbal tics: you know, I mean, of course; Avoids discussing: his past adventures, the burden of the ring

**Relationships:** Frodo: mentor and guardian (trust: 9/10, power: protective yet encourages independen